In [1]:
from PyMRStrain import *
import numpy as np
import matplotlib.pyplot as plt

if __name__=="__main__":

  # Parameters
  p = {'t_end': 1.0,
       'dt': 1.0/20.0,
       't': 0.0,
       'time_steps': 20,
       'mesh_resolution': 0.0001}

  # Encoding frequency
  ke = 1000*2*np.pi*0.125

  # Create image
  I0 = Image(np.array([0.15, 0.15, 0.2]),
            resolution=np.array([55, 55, 25]),
            center=np.array([0.01,-0.01,-0.03]),
            encoding_frequency=[ke,ke,ke],
            complimentary=False,
            T1=0.85,
            flip_angle=15*np.pi/180,
            type="DENSE")

  I1 = Image(np.array([0.15, 0.15, 0.2]),
            resolution=np.array([55, 55, 25]),
            center=np.array([0.01,-0.01,-0.03]),
            encoding_frequency=[ke,ke,ke],
            complimentary=True,
            T1=0.85,
            flip_angle=15*np.pi/180,
            type="DENSE")

  # Element
  FE = VectorElement('tetrahedron10')

  # Mesh and fem space
  mesh = Mesh('mesh/mesh10.msh')
  V = FunctionSpace(mesh, FE)

  # Create phantom object
  phantom = Phantom3D(function_space=V, time_steps=p['time_steps'], path='files/')

  # Generator
  g0 = Generator(p, I0, debug=True)
  g1 = Generator(p, I1, debug=True)

  # Generation
  g0.phantom = phantom
  g1.phantom = phantom
  u0, mask = g0.get_image()
  u1, mask = g1.get_image()

- Time: 0.00
- Time: 0.05


KeyboardInterrupt: 

In [3]:
  # Add noise to images
  un0 = add_noise_to_DENSE_(u0, mask, sigma=0.05)
  un1 = add_noise_to_DENSE_(u1, mask, sigma=0.05)  

  # Corrected image
  u = un0 - un1

  # Plot
  if rank==0:
    # fig, ax = plt.subplots(1, 2)
    # tracker = IndexTracker(ax, u[:,:,5,0,:], u[:,:,5,2,:])
    # fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
    # fig.suptitle('DENSE volunteer')
    # plt.show()

    # Export file
    export_image(u, path='u', name='I_DENSE')
    export_image(I0.Grid[0], path='X', name='X')
    export_image(I0.Grid[1], path='Y', name='Y')
    export_image(mask, path='mask', name='mask')